# mix-SQP demo with different low-rank matrix approximations

In this example, we illustrate how the QR and singular value decompositions of the likelihood matrix can be used to speed up the SQP algorithm. Compare these results to Fig. 2 of the paper.

## Analysis setup

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [6]:
using Distributions
using LowRankApprox
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/mixSQP_time.jl");

Next, initialize the sequence of pseudorandom numbers.

In [5]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [7]:
x = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [8]:
sd = autoselectmixsd(x,gridmult = 1.425);
L  = normlikmatrix(x,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm once to precompile the function.

In [12]:
out = mixSQP_time(L);

Next, run the mix-SQP solver again with the SVD and QR approximations to the likelihood matrix, and with no approximation. The approximation tolerance is set very low, to `1e-8`.

In [2]:
# pqrtol    : relative tolerance for randomized low-rank approximation
# lowrank : low-rank approximation method, perform randomized svd if "svd", randomized RRQR if "qr",
#           and do nothing otherwise

# example
data = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));
x_svd = mixSQP_time(data, pqrtol = 1e-8, lowrank = "svd")[1];
x_qr = mixSQP_time(data,  pqrtol = 1e-8, lowrank = "qr")[1];
x_nothing = mixSQP_time(data,  pqrtol = 1e-8, lowrank = "nothing")[1];

In [13]:
@time outSVD = mixSQP_time(L,lowrank = "svd");
@time outQR  = mixSQP_time(L,lowrank = "qr");
@time out    = mixSQP_time(L,lowrank = "nothing");

LoadError: [91mUndefVarError: data not defined[39m

In [4]:
data = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));

In [5]:
@time x_svd = mixSQP_time(data, lowrank = "svd")[1];
@time x_qr = mixSQP_time(data, lowrank = "qr")[1];
@time x_nothing = mixSQP_time(data, lowrank = "nothing")[1];
["rel_err btw x_svd,x" "rel_error btw x_qr,x"; rel_error(data,x_svd,x_nothing) rel_error(data,x_qr,x_nothing)]

  0.396186 seconds (56.12 k allocations: 324.077 MiB, 8.41% gc time)
  0.281044 seconds (8.66 k allocations: 321.349 MiB, 9.47% gc time)
  2.880684 seconds (6.51 k allocations: 2.412 GiB, 45.00% gc time)


2×2 Array{Any,2}:
  "rel_err btw x_svd,x"   "rel_error btw x_qr,x"
 0.0                     0.0                    

In [ ]:
L1 = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));
L2 = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));

In [ ]:
t = psvdfact(L1); D = (t[:U] .* t[:S]') * t[:Vt] - L1;
vecnorm(D),norm(D)

In [ ]:
t = psvdfact(L2); D = (t[:U] .* t[:S]') * t[:Vt] - L2;
vecnorm(D),norm(D)

In [ ]:
t = pqrfact(L1); D = t[:Q] * t[:R] * sparse(t[:P])' - L1;
vecnorm(D),norm(D)

In [ ]:
t = pqrfact(L2); D = t[:Q] * t[:R] * sparse(t[:P])' - L2;
vecnorm(D),norm(D)

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here.

In [6]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");

 - LowRankApprox                 0.1.0
 - CSV                           0.1.5


In [7]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
